In [1]:
from dotenv import load_dotenv

load_dotenv()

from ddtrace.llmobs import LLMObs

LLMObs.enable()

In [2]:
# https://github.com/DataDog/llm-observability/blob/main/3-agent-span.ipynb

system_prompt = """
You are a helpful assistant who can answer multistep questions by sequentially calling functions. 

Follow a pattern of:
THOUGHT (reason step-by-step about which function to call next),
ACTION (call a function to as a next step towards the final answer), 
OBSERVATION (output of the function).

Reason step by step which actions to take to get to the answer. 
Only call functions with arguments coming verbatim from the user or the output of other functions.
"""


def get_initial_messages(question_prompt):
    return [
        {
            "role": "system",
            "content": system_prompt,
        },
        {
            "role": "user",
            "content": question_prompt,
        },
    ]

In [3]:
import json
import requests
import time

FORECAST_API_URL = "https://api.open-meteo.com/v1/forecast"
CURRENT_LOCATION_BY_IP_URL = "http://ip-api.com/json?fields=lat,lon"


def get_current_location():
    time.sleep(0.5)  # simulate a longer task
    print(requests.get(CURRENT_LOCATION_BY_IP_URL).json())
    return json.dumps(requests.get(CURRENT_LOCATION_BY_IP_URL).json())


def get_current_weather(latitude, longitude, temperature_unit):
    time.sleep(0.3)  # simulate a longer task
    resp = requests.get(
        FORECAST_API_URL,
        params={
            "latitude": latitude,
            "longitude": longitude,
            "temperature_unit": temperature_unit,
            "current_weather": True,
        },
    )
    return json.dumps(resp.json())


def calculate(formula):
    return str(eval(formula))


class StopException(Exception):
    """
    Signal that the task is finished.
    """


def finish(answer):
    raise StopException(answer)


available_functions = {
    "get_current_location": get_current_location,
    "get_current_weather": get_current_weather,
    "calculate": calculate,
    "finish": finish,
}

In [4]:
function_schema = [
    {
        "function": {
            "name": "get_current_location",
            "description": "Get the current location of the user.",
            "parameters": {"type": "object", "properties": {}, "required": []},
        },
        "type": "function",
    },
    {
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location.",
            "parameters": {
                "type": "object",
                "properties": {
                    "latitude": {"type": "number"},
                    "longitude": {"type": "number"},
                    "temperature_unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                    },
                },
                "required": ["latitude", "longitude", "temperature_unit"],
            },
        },
        "type": "function",
    },
    {
        "function": {
            "name": "calculate",
            "description": "Calculate the result of a given formula.",
            "parameters": {
                "type": "object",
                "properties": {
                    "formula": {
                        "type": "string",
                        "description": "Numerical expression to compute the result of, in Python syntax.",
                    }
                },
                "required": ["formula"],
            },
        },
        "type": "function",
    },
    {
        "function": {
            "name": "finish",
            "description": "Once you have the information required, answer the user's original question, and finish the conversation.",
            "parameters": {
                "type": "object",
                "properties": {
                    "answer": {
                        "type": "string",
                        "description": "Answer to the user's question.",
                    }
                },
                "required": ["answer"],
            },
        },
        "type": "function",
    },
]

In [7]:
from ddtrace.llmobs.decorators import *
from openai import OpenAI

MAX_CALLS = 4
MODEL = "gpt-4o-mini"

client = OpenAI()

@workflow()
def execute_loop_step(messages, calls_left=MAX_CALLS):

    if calls_left < 1:
        return messages

    # https://platform.openai.com/docs/api-reference/chat/create#chat-create-messages
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=function_schema,
    )
    response_message = response.choices[0].message
    if response_message.content:
        print("\n")
        print(response_message.content)
    if response_message.tool_calls:
        print("\n")
        print("CALL TOOL:", response_message.tool_calls)
    messages.append(response_message)
    if not response_message.tool_calls:
        return execute_loop_step(messages, calls_left - 1)

    for tool_call in response_message.tool_calls:
        # define a small helper function to reduce repetitive code
        def append_tool_message_and_execute_loop(content):
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "content": content,
                }
            )
            return execute_loop_step(messages, calls_left - 1)

        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        if function_to_call is None:
            return append_tool_message_and_execute_loop(
                f"Invalid function name: {function_name!r}"
            )
        try:
            function_args_dict = json.loads(tool_call.function.arguments)
        except json.JSONDecodeError as exc:
            return append_tool_message_and_execute_loop(
                f"Error decoding function call `{function_name}` arguments {tool_call.function.arguments!r}! Error: {exc!s}"
            )
        try:
            with LLMObs.tool(function_name):
                LLMObs.annotate(input_data=function_args_dict)
                try:
                    function_response = function_to_call(**function_args_dict)
                    LLMObs.annotate(output_data=function_response)
                except StopException as answer:
                    LLMObs.annotate(output_data="StopException")
                    return str(answer)
            return append_tool_message_and_execute_loop(function_response)
        except Exception as exc:
            return append_tool_message_and_execute_loop(
                f"Error calling function `{function_name}`: {type(exc).__name__}: {exc!s}!"
            )
    return "no answer found"

In [8]:
# https://docs.datadoghq.com/tracing/llm_observability/sdk/#agent-span
@agent()
def call_weather_assistant(question_prompt):
    LLMObs.annotate(
        input_data=question_prompt,
    )
    messages = get_initial_messages(question_prompt)
    answer = execute_loop_step(messages)
    LLMObs.annotate(
        output_data=answer,
    )
    return answer


call_weather_assistant(
    "What is the weather in my current location? Please give me the temperature in farenheit. Also tell me my current location coordinates."
)



CALL TOOL: [ChatCompletionMessageToolCall(id='call_Bvp7ypyxWNUUWvyTdKajANcs', function=Function(arguments='{}', name='get_current_location'), type='function')]
{'lat': 37.5794, 'lon': 126.9754}


I/O object is not JSON serializable. Defaulting to placeholder value instead.




CALL TOOL: [ChatCompletionMessageToolCall(id='call_vlpQveEzH6cDLvfMa6j37onW', function=Function(arguments='{"latitude":37.5794,"longitude":126.9754,"temperature_unit":"fahrenheit"}', name='get_current_weather'), type='function')]


CALL TOOL: [ChatCompletionMessageToolCall(id='call_7mNhCFqTeApzttssVVbUpYrR', function=Function(arguments='{"answer":"Your current location coordinates are Latitude: 37.5794, Longitude: 126.9754. The temperature in your current location is 43.3°F."}', name='finish'), type='function')]


'Your current location coordinates are Latitude: 37.5794, Longitude: 126.9754. The temperature in your current location is 43.3°F.'